In [ ]:
%serialconnect to port=/dev/tty.usbserial-0001 --baud=115200

Found serial ports: /dev/cu.usbserial-0001, /dev/cu.Bluetooth-Incoming-Port, /dev/cu.wlan-debug 
Connecting to --port=/dev/cu.usbserial-0001 --baud=115200 


In [ ]:
from machine import Pin, SoftI2C
from time import sleep_ms

import os

mpu_addr = 0x68
mpu_pwr_mgmt = 0x6B

mpu_accel_x_h = 0x3B
mpu_accel_x_l = 0x3C
mpu_accel_y_h = 0x3D
mpu_accel_y_l = 0x3E
mpu_accel_z_h = 0x3F
mpu_accel_z_l = 0x40

mpu_gyro_x_h = 0x43
mpu_gyro_x_l = 0x44
mpu_gyro_y_h = 0x45
mpu_gyro_y_l = 0x46
mpu_gyro_z_h = 0x47
mpu_gyro_z_l = 0x48

mpu_lsbc = 340.0
mpu_lsbg = 16348.0
mpu_lsbds = 131.0

def mpu_init(i2c):
    i2c.writeto_mem(mpu_addr, mpu_pwr_mgmt, bytes([0]))

def combine_register_values(h, l):
    if not h[0] & 0x80:
        return h[0] << 8 | l[0]
    return -((h[0] ^ 255) << 8) |  (l[0] ^ 255) + 1

def mpu_get_accel(i2c):
    accel_x_h = i2c.readfrom_mem(mpu_addr, mpu_accel_x_h, 1)
    accel_x_l = i2c.readfrom_mem(mpu_addr, mpu_accel_x_l, 1)
    accel_y_h = i2c.readfrom_mem(mpu_addr, mpu_accel_y_h, 1)
    accel_y_l = i2c.readfrom_mem(mpu_addr, mpu_accel_y_l, 1)
    accel_z_h = i2c.readfrom_mem(mpu_addr, mpu_accel_z_h, 1)
    accel_z_l = i2c.readfrom_mem(mpu_addr, mpu_accel_z_l, 1)

    return [combine_register_values(accel_x_h, accel_x_l) / mpu_lsbg,
            combine_register_values(accel_y_h, accel_y_l) / mpu_lsbg,
            combine_register_values(accel_z_h, accel_z_l) / mpu_lsbg]

def mpu_get_gyro(i2c):
    gyro_x_h = i2c.readfrom_mem(mpu_addr, mpu_gyro_x_h, 1)
    gyro_x_l = i2c.readfrom_mem(mpu_addr, mpu_gyro_x_l, 1)
    gyro_y_h = i2c.readfrom_mem(mpu_addr, mpu_gyro_y_h, 1)
    gyro_y_l = i2c.readfrom_mem(mpu_addr, mpu_gyro_y_l, 1)
    gyro_z_h = i2c.readfrom_mem(mpu_addr, mpu_gyro_z_h, 1)
    gyro_z_l = i2c.readfrom_mem(mpu_addr, mpu_gyro_z_l, 1)

    return [combine_register_values(gyro_x_h, gyro_x_l) / mpu_lsbg,
            combine_register_values(gyro_y_h, gyro_y_l) / mpu_lsbg,
            combine_register_values(gyro_z_h, gyro_z_l) / mpu_lsbg]

if __name__ == "__main__":
    i2c = SoftI2C(scl=Pin(23), sda=Pin(22))
    mpu_init(i2c)
    
    while True:
        print("Accelerometer:\t", mpu_get_accel(i2c), "g")
        print("Gyroscope:\t", mpu_get_gyro(i2c), "°/s")
        sleep_ms(1)
